### This is an attempt to convert the colab cloud version of exotic to a local jupyter notebook
It can be now used to:
- compare a fits image with star chart
- find the coordinates of the stars
- clean the images

## This notebook can be used as a fits viewer of all the images and find the coordinates required for target and comparison stars.

Instead of using any fits viewer to find the coordinates of the target star and comparison stars, you can use this notebook by running it locally to find the fits image and star chart for comparison and gather the coordinates.

You can also check all the fits images here and do cleaning if required

Use local EXOTIC along with this as this notebook doesn't do beyond step 3 of whats there in colab standard EXOTIC version.

---

## **Identify target star and comparison stars**

<font face="Helvetica, Arial, Sans-Serif">


🕔&nbsp;&nbsp;Estimated time: 1 minutes
<br />

<br />


This step will:

<ol type="a">
<li class="step">Enter your telescope specifications (typically "MicroObservatory" for Standard users).</li>
<li class="step">Enter your target host star (omit the "b" or other trailing letter from your object name).</li>
<li class="step">Ask you to identify coordinates for your target star and comparison stars</li>
</ol>


For more information:

<ul>
<li class="step"><a href="https://exoplanets.nasa.gov/exoplanet-watch/exotic/user-guide/#2-2">How do I get a starchart image URL?</a></li>
<li class="step"><a href="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.html" target="_blank">How do I identify coordinates for my target and comparison stars?</a>
</li>
</ul>

</font>

In [33]:
import os
import re
import json
import urllib.request
from urllib.error import HTTPError
from astropy.io import fits
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import LogColorMapper, ColorBar, HoverTool, CrosshairTool, PanTool, BoxZoomTool, WheelZoomTool, ResetTool, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.models.widgets import Div
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256
from IPython.display import display, HTML

output_notebook()

### 1. **Set up Telescope Parameters**
def get_star_chart_urls(telescope, star_target):
    # Telescope parameters for field of view, magnitude limit, and resolution
    t_fov, t_maglimit, t_resolution = 56.44, 15, 150  # Default values

    if telescope == 'MicroObservatory':
        t_fov = 56.44
    elif telescope == 'Exoplanet Watch .4 Meter':
        t_fov = 38.42

    # Construct URLs for the star chart
    json_url = f"https://app.aavso.org/vsp/api/chart/?star={star_target}&scale=D&orientation=CCD&type=chart&fov={t_fov}&maglimit={t_maglimit}&resolution={t_resolution}&north=down&east=left&lines=True&format=json"
    starchart_url = f"https://app.aavso.org/vsp/?star={star_target}&scale=D&orientation=CCD&type=chart&fov={t_fov}&maglimit={t_maglimit}&resolution={t_resolution}&north=down&east=left&lines=True"
    
    return [json_url, starchart_url]

def get_star_chart_image_url(json_url):
    display(HTML(f'<p>Searching for star chart at {json_url}</p>'))
    with urllib.request.urlopen(json_url) as url:
        starchart_data = json.load(url)
        image_uri = starchart_data["image_uri"].split('?')[0]
        display(HTML(f'<p>Found image URL: {image_uri}</p>'))
        return image_uri

### 2. **Load and Display the FITS Image with Bokeh**
def display_image_with_bokeh(fits_file):
    # Load the FITS file
    hdul = fits.open(fits_file)
    data = hdul[0].data

    # Check for multi-extension FITS and select the first one
    if data.ndim > 2:
        data = data[0]  # In case of multi-extension FITS

    # 3. **Downsampling the Image (Optional)**
    megapixel_factor = (data.shape[0]) * (data.shape[1]) / 1000000.0
    if megapixel_factor > 5:
        print(f"Downsampling because it has {megapixel_factor} megapixels.")
        data = data[::2, ::2]  # Downsample by selecting every second pixel

    # 4. **Setting Plot Size Dynamically**
    target_width, target_height = 1230, 1530
    #data = data[:target_height, :target_width]  # Crop the data to fit the dimensions
    # Set plot dimensions to match AAVSO chart (1230x1530)
    #p_width, p_height = target_width, target_height
    p_height = 500
    max_y, max_x = data.shape
    p_width = int((p_height / max_y) * max_x)

    # 5. **Color Mapping (Logarithmic Scale)**
    color_mapper = LogColorMapper(palette="Cividis256", low=np.percentile(data, 5), high=np.percentile(data, 99))

    # 6. **Creating the Plot (`figure`)**
    p = figure(
        title="FITS Image",
        width=p_width, height=p_height,
        tools=[PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool(), HoverTool()],
        tooltips=[("x", "$x"), ("y", "$y")],  # Temporarily remove the "value" tooltip
        x_range=(0, max_x), y_range=(0, max_y),
        match_aspect=True
    )


    # 7. **Displaying the Image**
    p.image(image=[data], x=0, y=0, dw=max_x, dh=max_y, color_mapper=color_mapper)

    # 8. **Adding Interactive Crosshair Tool**
    crosshair = CrosshairTool(dimensions="both")
    p.add_tools(crosshair)

    # 9. **Adding a Color Bar**
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')

    return p

### 3. **Main Routine to Guide the User and Compare Star Chart with FITS Image**
def main():
    # User input for telescope and star
    Telescope = input("Select your telescope (MicroObservatory or Exoplanet Watch .4 Meter): ")
    Target = input("Enter your target star (e.g., 'HAT-P-32'): ")

    if not Target.strip() or Telescope not in ["MicroObservatory", "Exoplanet Watch .4 Meter"]:
        display(HTML('<span class="error">You must select a valid Telescope and Target Star.</span>'))
        return

    # Generate Star Chart URLs
    starchart_urls = get_star_chart_urls(Telescope, Target.strip())
    try:
        # Retrieve star chart image URL
        starchart_image_url = get_star_chart_image_url(starchart_urls[0])
    except HTTPError:
        display(HTML(f'<p class="error">Could not find a star chart for {Target}. Try a different target or use <a href="{starchart_urls[1]}" target="_blank">AAVSO advanced search</a>.</p>'))
        return

    # Load the FITS image (first one in the directory)
    fits_dir = input("Enter the directory path containing your FITS files: ")
    fits_files = [f for f in os.listdir(fits_dir) if f.endswith('.fits') or f.endswith('.FITS.gz')]
    first_image = fits_files[0] if fits_files else None
    
    
    if first_image:
        first_image_path = os.path.join(fits_dir, first_image)
        print(first_image_path)
        # Display the star chart image alongside the FITS image
        fits_plot = display_image_with_bokeh(first_image_path)
        
        # Bokeh layout with star chart and FITS image
        img_div = Div(text=f'<img src="{starchart_image_url}" alt="Star Chart" style="width:40%; height: auto;">')

        layout = row(fits_plot, img_div)
        
        # Show the combined layout
        show(layout)
        # Guide the user to enter star coordinates
        target_coords = input("Enter the coordinates for the target star [x, y]: ")
        comparison_coords = input("Enter the coordinates for the comparison stars [[x1, y1], [x2, y2], ...]: ")
        print(f'\n\ntarget_coords:{target_coords}\ncomparison_coords:{comparison_coords}')
        
        # Basic validation for coordinate format
        if re.match(r"\[\d+, ?\d+\]$", target_coords) and re.match(r"\[(\[\d+, ?\d+\],? ?)+\]$", comparison_coords):
            display(HTML('<p>Coordinates saved successfully.</p>'))
        else:
            display(HTML('<p class="error">Invalid coordinate format. Please retry.</p>'))

    else:
        display(HTML('<p class="error">No FITS files found in the directory. Please check the path and try again.</p>'))
   
# Run the main routine
main() 


Loading BokehJS ...

Select your telescope (MicroObservatory or Exoplanet Watch .4 Meter):  MicroObservatory
Enter your target star (e.g., 'HAT-P-32'):  Qatar-6


Enter the directory path containing your FITS files:  /media/vaishnav/DATA/Projects/EXOPLANET WATCH/2nd/b9186335455028298291bb6f54f1e606


MOBS_Cecilia_Qatar-10_2022-04-20T23_06_24.847-0700_8478fd022189d902d44a2c1c75454cad_Qatar-10220421060624.FITS.gz
/media/vaishnav/DATA/Projects/EXOPLANET WATCH/2nd/b9186335455028298291bb6f54f1e606/MOBS_Cecilia_Qatar-10_2022-04-20T23_06_24.847-0700_8478fd022189d902d44a2c1c75454cad_Qatar-10220421060624.FITS.gz


Enter the coordinates for the target star [x, y]:  dd
Enter the coordinates for the comparison stars [[x1, y1], [x2, y2], ...]:  d




target_coords:dd
comparison_coords:d


## Cleaning(optional step)

This optional step allows you to visually inspect every image in your dataset to then manually remove any bad images (e.g., due to clouds).

Note: This step can take some time to run as it takes ~1 second to load each image. If you run into issues, remember that EXOTIC Standard is meant to run just one dataset each time. Try going to Runtime > Disconnect and Delete Runtime, and start over for each new dataset. Report issues to our #exotic Slack Channel (link above).

In [9]:
# IMAGE CLEANING STEP

import os
import shutil
from io import BytesIO
import ipywidgets as widgets
from ipywidgets import VBox, HBox, GridBox
from astropy.io import fits
from astropy.visualization import ImageNormalize, ZScaleInterval
import numpy as np
import matplotlib.pyplot as plt


local_directory_filepath = input("First tell me the directory path where your fits files are located:")
sorted_files = sorted(os.listdir(local_directory_filepath));

#redefine fits_list so you can run this every time even after you've removed images
uploaded_files = [f for f in sorted_files if os.path.isfile(os.path.join(local_directory_filepath, f))]
#fits_count, inits_count, first_image = 0, 0, ""

# Identify .FITS and inits.json files in user-submitted folder
#inits = []    # array of paths to any inits files found in the directory
fits_list = []
first_image = ""
for f in uploaded_files:
  # Look for .fits images and keep count
  if f.lower().endswith(('.fits', '.fits.gz', '.fit')):
    fits_list.append(f)
    if first_image == "":
      first_image = os.path.join(local_directory_filepath, f)

def display_all_fits_with_remove(directory_path, fits_files):
    """
    Function that displays all FITS images simultaneously in a grid layout,
    with a toggle button for each image to remove or undo.
    Displays a loading progress bar with percentage and removes it on 'Done'.

    Parameters:
    directory_path (str): Path to the directory containing the FITS files.
    fits_files (list): List of FITS files to be reviewed.

    Returns:
    None
    """
    total_images = len(fits_files)
    image_count = widgets.FloatProgress(value=0, min=0, max=total_images, description='Loading:', layout=widgets.Layout(width='50%'))
    percentage_label = widgets.Label(value='0%', layout=widgets.Layout(width='50%'))
    loading_progress = HBox([image_count, percentage_label])
    display(loading_progress)

    # Make 'Bad Images' directory if it doesn't exist
    bad_dir = os.path.join(directory_path, 'Bad Images')
    os.makedirs(bad_dir, exist_ok=True)

    # Create an output area for messages
    output_area = widgets.Output()

    # Define the button click actions
    def update_progress():
        percentage = int(image_count.value / total_images * 100)
        percentage_label.value = f'{percentage}%'

    def on_toggle_button_clicked(b, i):
        image_path = os.path.join(directory_path, fits_files[i])
        bad_image_path = os.path.join(bad_dir, fits_files[i])

        if os.path.exists(bad_image_path):
            # Undo the removal
            shutil.move(bad_image_path, image_path)
            with output_area:
                output_area.clear_output(wait=True)
                display(HTML(f"<p class='output'>Image {i + 1} has been restored.</p>"))
            b.button_style = 'danger'
            b.description = 'Remove'

            if mass_removal_mode.value:
                # Restore subsequent images
                for j in range(i + 1, total_images):
                    subsequent_image_path = os.path.join(directory_path, fits_files[j])
                    subsequent_bad_image_path = os.path.join(bad_dir, fits_files[j])
                    if os.path.exists(subsequent_bad_image_path):
                        shutil.move(subsequent_bad_image_path, subsequent_image_path)
                        toggle_buttons[j].button_style = 'danger'
                        toggle_buttons[j].description = 'Remove'
                with output_area:
                    output_area.clear_output(wait=True)
                    display(HTML(f"<p class='output'>All images from Image {i + 1} to Image {total_images} have been restored.</p>"))

        else:
            # Remove the image
            shutil.move(image_path, bad_image_path)
            with output_area:
                output_area.clear_output(wait=True)
                display(HTML(f"<p class='output'>Image {i + 1} has been removed.</p>"))
            b.button_style = 'info'
            b.description = 'Restore'

            if mass_removal_mode.value:
                # Remove previous images
                for j in range(i - 1, -1, -1):
                    previous_image_path = os.path.join(directory_path, fits_files[j])
                    previous_bad_image_path = os.path.join(bad_dir, fits_files[j])
                    if not os.path.exists(previous_bad_image_path):
                        shutil.move(previous_image_path, previous_bad_image_path)
                        toggle_buttons[j].button_style = 'info'
                        toggle_buttons[j].description = 'Restore'
                with output_area:
                    output_area.clear_output(wait=True)
                    display(HTML(f"<p class='output'>All images from Image {1} to Image {i + 1} have been removed.</p>"))

    # Create image display areas and toggle buttons for each image
    image_areas = []
    toggle_buttons = []
    for i, fits_file in enumerate(fits_files):
        # Image display area
        image_area = widgets.Output()

        # Toggle button with "Remove" initial description and 75% width
        toggle_button = widgets.Button(description='Remove', button_style='danger', layout=widgets.Layout(width='75%'))
        toggle_button.on_click(lambda b, i=i: on_toggle_button_clicked(b, i))

        # Store references
        image_areas.append(image_area)
        toggle_buttons.append(toggle_button)

        # Display the image
        with fits.open(os.path.join(directory_path, fits_file)) as hdul:
            image_data = hdul[0].data
            norm = ImageNormalize(image_data, interval=ZScaleInterval(), vmin=np.nanpercentile(image_data, 5),
                                  vmax=np.nanpercentile(image_data, 99))
            fig, ax = plt.subplots()
            ax.imshow(image_data, cmap='viridis', norm=norm)
            ax.axis('off')
            ax.set_title(f"Image {i + 1}/{total_images}")

            # Convert the plot to an image
            with BytesIO() as output:
                fig.savefig(output, dpi=100, bbox_inches='tight', pad_inches=0, format='png', transparent=True)
                image_bytes = output.getvalue()
            plt.close(fig)

            # Display the image in the widget
            with image_area:
                image_area.clear_output(wait=True)
                display(widgets.Image(value=image_bytes))

        # Update loading progress after displaying each image
        image_count.value += 1
        update_progress()

    # Mass removal mode toggle
    mass_removal_mode = widgets.ToggleButton(value=False, description='Enable Mass Removal Mode', button_style='success', layout=widgets.Layout(width='30%'))
    mass_removal_message = widgets.HTML(value='')

    def toggle_mass_removal_mode(change):
        if change['new']:
            mass_removal_mode.description = 'Disable Mass Removal Mode'
            mass_removal_mode.button_style = 'danger'
            mass_removal_message.value = "<p style='color:red;'>Mass removal mode is enabled. Clicking 'Remove' will remove the selected image and all previous images. Clicking 'Restore' will restore the selected image and all subsequent images.  Most EXOTIC runs only need the first image to be good (to find the coordinates); it will automatically filter out the rest itself.</p>"
        else:
            mass_removal_mode.description = 'Enable Mass Removal Mode'
            mass_removal_mode.button_style = 'success'
            mass_removal_message.value = ''

    mass_removal_mode.observe(toggle_mass_removal_mode, 'value')

    # Create a grid layout
    num_columns = 3
    num_rows = (total_images + num_columns - 1) // num_columns
    grid_items = [widgets.VBox([image_areas[i], toggle_buttons[i]]) for i in range(total_images)]
    grid_layout = widgets.GridBox(grid_items, layout=widgets.Layout(
        grid_template_columns=f"repeat({num_columns}, 1fr)",
        gap='20px'
    ))

    # Done button
    done_button = widgets.Button(description='Done', button_style='warning', layout=widgets.Layout(width='20%'))
    def on_done_button_clicked(b):
        loading_progress.close()
        output_area.clear_output()
        grid_layout.close()
        done_button.close()
        mass_removal_mode.close()
        mass_removal_message.close()

    done_button.on_click(on_done_button_clicked)

    # Display the mass removal mode toggle, message, grid layout, and output area
    display(mass_removal_mode)
    display(mass_removal_message)
    display(grid_layout)
    display(widgets.HBox([done_button, output_area]))

display_all_fits_with_remove(local_directory_filepath, fits_list)



First tell me the directory path where your fits files are located: /media/vaishnav/DATA/Projects/EXOPLANET WATCH/2nd/b9186335455028298291bb6f54f1e606


ToggleButton(value=False, button_style='success', description='Enable Mass Removal Mode', layout=Layout(width=…

HTML(value='')

GridBox(children=(VBox(children=(Output(), Button(button_style='danger', description='Remove', layout=Layout(w…

### FITS File info

If you want RA-Dec values, observation date and any other header values of the fits file, just keep your fits file path ready and run the following cell:

In [38]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from datetime import datetime

fits_file = input("Input the path to the fits file:")
# Open the FITS file and extract the RA and Dec from the header
with fits.open(fits_file) as hdul:
    header = hdul[0].header  # Access the primary header
    ra_deg = header['RA']     # Get RA from the header (in degrees)
    dec_deg = header['DEC']   # Get Dec from the header (in degrees)
    # Get the observation date
    date_obs_str = header['DATE-OBS']  # e.g., '2022-04-21T00:48:14.626-0700'
    

# Convert RA and Dec to HH:MM:SS and DD:MM:SS format
coord = SkyCoord(ra=ra_deg*u.degree, dec=dec_deg*u.degree)

# Convert to the desired string formats
ra_hms = coord.ra.to_string(u.hour, sep=':', precision=2)
dec_dms = coord.dec.to_string(sep=':', alwayssign=True, precision=2)

# Display the results
print("\nThe RA-Dec values corresponding to the fit file is\n")
print(f"RA (degrees): {ra_deg} -> RA (HH:MM:SS): {ra_hms}")
print(f"Dec (degrees): {dec_deg} -> Dec (DD:MM:SS): {dec_dms}\n\n")


# Convert the date to DD-MM-YYYY format
date_obs = datetime.strptime(date_obs_str, '%Y-%m-%dT%H:%M:%S.%f%z')
formatted_date = date_obs.strftime('%d-%m-%Y')
#Display the result
print("Observation Date:", formatted_date)

# Convert header to a table format
header_table = Table(rows=header.items(), names=['Keyword', 'Value'])
print("\nThe header table for the fits file:")
# Show the header table
header_table.show_in_notebook()


Input the path to the fits file: /media/vaishnav/DATA/Projects/EXOPLANET WATCH/2nd/b9186335455028298291bb6f54f1e606/MOBS_Cecilia_Qatar-10_2022-04-20T23_06_24.847-0700_8478fd022189d902d44a2c1c75454cad_Qatar-10220421074814.FITS.gz



The RA-Dec values corresponding to the fit file is

RA (degrees): 222.458547 -> RA (HH:MM:SS): 14:49:50.05
Dec (degrees): 22.061747 -> Dec (DD:MM:SS): +22:03:42.29


Observation Date: 21-04-2022

The header table for the fits file:


idx,Keyword,Value
0,SIMPLE,True
1,BITPIX,16
2,NAXIS,2
3,NAXIS1,650
4,NAXIS2,500
5,EQUINOX,2000.0
6,FILENAME,Qatar-10220421074814.FIT
7,DATE-OBS,2022-04-21T00:48:14.626-0700
8,DATE-END,2022-04-21T00:49:16.586-0700
9,LST-OBS,07:21:02


In [ ]:
#Thanks to ChatGPT and EXOTIC team who made developing this an easy task
#Vai838